# Data Preprocessing and Cleaning

Data cleaning is an essential step in the process of preparing data for further processing and serving into a large language model. Without data cleaning, the language model would be limited in its ability to interpret and process the data accurately. Data cleaning helps to ensure that the data is in a consistent format, free of any inconsistencies or errors that could lead to incorrect results. It also helps to reduce the amount of time needed to process the data, as it eliminates the need to manually check for any inconsistencies or errors. Additionally, data cleaning can help to improve the accuracy of the language model by removing any irrelevant or incorrect data. By taking the time to properly clean the data, the language model can more accurately interpret and process the data, leading to better results.

> 📍 Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

## Count tokens

> **Important note**
>
> *You do not need to manually tokenize strings before feeding texts into the model. This will be done automatically once you put instructions into the `prompt` parameter. However, you can use the `tiktoken` library to check how a string is tokenized and count the numbers of tokens to calculate the cost of an API call. Learn more [here](https://platform.openai.com/docs/introduction/tokens).*

Tokenizing text strings is an important step in natural language processing (NLP) as it helps models like GPT-3 understand the structure of a text string. Tokenizing breaks a text string into smaller pieces called tokens, which can then be analyzed and used by the model. By understanding the structure of a text string, models can better understand the meaning of the text. Additionally, tokenizing helps to determine the cost of an Azure OpenAI Service API call, as usage is priced by token. Furthermore, different models use different encodings, so it is important to tokenize text strings in the appropriate format.

`tiktoken` supports three encodings used by Azure OpenAI Service models:

| Encoding name | Azure OpenAI Service models |
| ------------- | -------------- |
| gpt2 (or r50k_base) | Most GPT-3 models |
| p50k_base | Code models, text-davinci-002, text-davinci-003 |
| cl100k_base | text-embedding-ada-002 |

Tokens in English typically range from one character to one word (e.g. "t" or "great"), though some languages may have tokens that are shorter or longer than one character or word. Spaces are usually placed at the beginning of words (e.g. " is" instead of "is " or "+"is"). You can use the Tokenizer to quickly check how a string is tokenized.

To show it briefly, we will use `tiktoken` to tokenize a text string and see how the output looks like.

In [1]:
! pip install tiktoken

In [44]:
import tiktoken

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode("Hello world, this is fun!")

[15496, 995, 11, 428, 318, 1257, 0]

Write a script that shows the string tokens from an input phrase.

In [45]:
codificador = tiktoken.get_encoding("cl100k_base")

texto = "Hello World, this is fun!"

tokens = codificador.encode(texto)

tokens_strings = [codificador.decode([token]) for token in tokens]

print("Texto original:", texto)
print("Tokens:", tokens)
print("Tokens como strings:", tokens_strings)

Texto original: Hello World, this is fun!
Tokens: [9906, 4435, 11, 420, 374, 2523, 0]
Tokens como strings: ['Hello', ' World', ',', ' this', ' is', ' fun', '!']


Let's write a function to count the number of tokens in a text string.

In [46]:

def get_num_tokens_from_string(string: str, encoding_name: str='p50k_base') -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

# Example usage
print(get_num_tokens_from_string("Hello World, this is fun!"))


7


## Clean data

Next we'll perform some light data cleaning by removing redundant whitespace and cleaning up the punctuation to prepare the data for tokenization. Again, tokenization is not required for the model to work, but it is a good practice to do so to ensure that the data is in a consistent format and that the model is able to process the data correctly. Also, it makes sure that the request is not too long for the model as the maximum number of tokens for davinci is 2048, e.g., equivalent to around 2-3 pages of text.

> **Best Practices**
> - **Replace newlines with a single space**: Unless you're embedding code, we suggest replacing newlines (\n) in your input with a single space, as we have observed inferior results when newlines are present.

In [47]:
import re

def normalize_text(string, sep_token = " \n "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # make text lowercase
    string = re.sub(r'\s+',  ' ', string).strip()
    string = re.sub(r". ,","",string)
    # remove all instances of multiple spaces
    string = string.replace("..",".")
    string = string.replace(". .",".")
    string = string.replace("\n", "")
    string = string.strip()
    return string

Generate some data to test the cleaning function.

In [48]:

text = " This is   a text with inconsistent spacing and.. punctuation."
print(normalize_text(text))  # Cleaned and normalized text


This is a text with inconsistent spacing and. punctuation.


## Ejercicio 1: Validar longitud de texto antes de enviar a OpenAI

Escribe una función llamada validate_text_length que valide si un texto supera un límite de tokens especificado. Si el texto supera el límite, la función debe dividirlo en segmentos de tamaño adecuado para enviarlos al modelo de OpenAI. Usa la codificación cl100k_base.



In [49]:

import tiktoken

def validate_text_length(text, max_tokens=2048, encoding_name='cl100k_base'):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    segments = []
    for i in range(0, len(tokens), max_tokens):
        segments.append(encoding.decode(tokens[i:i+max_tokens]))
    return segments

text = "Lorem ipsum " * 500
segments = validate_text_length(text)
print(len(segments))
print(len(tiktoken.get_encoding('cl100k_base').encode(segments[0])))


1
1001


## Ejercicio 2: Filtrar palabras prohibidas en un texto

Escribe una función llamada filter_prohibited_words que remueva palabras prohibidas de un texto antes de enviarlo a OpenAI. 

Como ejemplo queremos Eliminar palabras como: ['password', 'confidential', 'secret']. 



In [2]:

def filter_prohibited_words(text, prohibited_words):
    pattern = re.compile(r"\b(" + "|".join(re.escape(word) for word in prohibited_words) + r")\b", re.IGNORECASE)
    return pattern.sub("[Oculto]", text)

text = "El código es: secreto"
prohibited_words = ['secreto']
print(filter_prohibited_words(text, prohibited_words))


El código es: [Oculto]


## Ejercicio 3: Identificar idioma del texto

Crea una función detect_language que use langdetect para identificar el idioma de un texto y devuelva el código del idioma (e.g., en, es). Verifica el idioma antes de enviarlo al modelo OpenAI. Debes instalar la libreria langdetect


In [51]:
! pip install langdetect

In [52]:

from langdetect import detect

def detect_language(text):
    """Detects the language of a given text."""
    return detect(text)

text = "Hola, este es un texto en español."
print(detect_language(text))


es


## Ejercicio 5: Generar prompts efectivos

Escribe una función generate_effective_prompt que genere un prompt optimizado para OpenAI, dado un tema y un objetivo. Usa plantillas como:
“Explain [tema] in simple terms for a [audiencia].”



In [53]:

def generate_effective_prompt(topic, audience="general audience"):
    """Genera un prompt optimizado para OpenAI."""
    return f"Explica {topic} de manera simple para {audience}."

topic = "machine learning"
print(generate_effective_prompt(topic, "principiantes"))  


Explain machine learning in simple terms for a beginner.


## Ejercicio 6: Medir costo estimado del uso de OpenAI

Crea una función estimate_cost que calcule el costo estimado de una solicitud a OpenAI basado en el número de tokens. Supón un costo de $0.02 por 1000 tokens.



In [54]:

def estimate_cost(text, encoding_name='cl100k_base', cost_per_1k_tokens=0.02):
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(text))
    return (num_tokens / 1000) * cost_per_1k_tokens

text = "This is a sample text." * 100
print(estimate_cost(text))  


0.01002


## Ejercicio 7: Limpiar texto JSON para OpenAI

Escribe una función clean_json_text que tome un JSON como entrada y limpie todas las claves y valores de caracteres no ASCII.


In [1]:

import re

def clean_json_text(json_obj):
    """Limpia texto JSON eliminando caracteres no ASCII."""
    def limpiar_caracter(caracter):
        if isinstance(caracter, str):
            return re.sub(r'[^\x00-\x7F]', '', caracter)
        
        elif isinstance(caracter, dict):
            return {k: limpiar_caracter(v) for k, v in caracter.items()}
        
        elif isinstance(caracter, list):
            return [limpiar_caracter(item) for item in caracter]
        
        return caracter

    return {k: limpiar_caracter(v) for k, v in json_obj.items()}

data = {"key1": "Buenas 😊", "key2": "Café y té", "nested": {"key3": "¡Hola mundo! 🌍"}}
print(clean_json_text(data))



{'key1': 'Buenas ', 'key2': 'Caf y t', 'nested': {'key3': 'Hola mundo! '}}
